[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Anutri03/ML-AI-prep/blob/main/NLP/TrueFakenewsclassification.ipynb)


In [1]:
!pip install -q scikit-learn

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re # regular expression

In [3]:
import nltk
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM ,Dense , Dropout
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report


In [4]:
nltk.download('stopwords')
nltk.download('punkt_tab')

from nltk.corpus import stopwords
stopwords = stopwords.words('english')
from nltk.tokenize import word_tokenize

print("libraries loaded")
#print it

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...


libraries loaded


[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [5]:
!wget -q https://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip


In [9]:
embedding_index = {}
with open('glove.6B.100d.txt',encoding='utf-8') as f:
  for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float32')
    embedding_index[word] = coefs

print(f'loaded {len(embedding_index)} word vectors.')

loaded 400000 word vectors.


In [10]:
true_df = pd.read_csv('True.csv')
fake_df = pd.read_csv('Fake.csv')

In [13]:
true_df.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [11]:
true_df['label'] = 1
fake_df['label'] = 0

In [15]:
data = pd.concat([true_df,fake_df],axis=0).reset_index(drop=True)
data.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [16]:
data.shape

(44898, 5)

In [21]:
def clean_text(text):
  text = text.lower()
  text = re.sub(r'[^a-zA-Z0-9\s]','',text)
  text = re.sub(r'\W',' ',text)
  text = re.sub(r'\s+',' ',text) # remove extra spaces
  text = re.sub(r'\d',' ',text)
  word = text.split()
  word = [w for w in word if w not in stopwords]
  text = ' '.join(word)
  return text

data['text'] = data['text'].apply(clean_text)
data.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",washington reuters head conservative republica...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,washington reuters transgender people allowed ...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,washington reuters special counsel investigati...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,washington reuters trump campaign adviser geor...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,seattlewashington reuters president donald tru...,politicsNews,"December 29, 2017",1


In [23]:
# tokeniztion and padding
max_words = 5000
maX_len = 200

tokenizer = Tokenizer(num_words=max_words) #tokenizer model
tokenizer.fit_on_texts(data['text']) # fit train on the text data
sequences = tokenizer.texts_to_sequences(data['text']) # into seq
padded_sequences = pad_sequences(sequences,maxlen=maX_len) # seq len

X = padded_sequences
y = data['label'].values


In [26]:
len(X)

44898

In [27]:
len(y)

44898